# Explainable Embeddings

In [27]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('C:\\Users\\isabe\\Documents\\AI studies\\6.Semester\\Bachelor Thesis\\Paradime-NAMs\\Datasets\\heart.csv')
df.info

<bound method DataFrame.info of      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1    

In [15]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [16]:
df = df.dropna()
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [33]:
input_dim = len(df.columns) - 1 # minus target

13

In [18]:
def preprocess_dataframe(df):
    # Iterate over each column in the DataFrame
    for column in df.columns:
        if len(df[column].unique()) == 2 and df[column].dtype == object:
            # Binary labels
            unique_labels = df[column].unique()
            mapping = {unique_labels[0]: 0, unique_labels[1]: 1}
            df[column] = df[column].map(mapping)
        elif df[column].dtype == object:
            # Multiple labels (strings)
            dummies = pd.get_dummies(df[column], prefix=column)
            df = pd.concat([df, dummies], axis=1)
            df.drop(column, axis=1, inplace=True)

    return df

In [19]:
clean_data = preprocess_dataframe(df)
clean_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [20]:
# Selecting feature vector and target variable
target_column = 'target' # should be specified by user 
X = df.drop([target_column], axis=1)
y = df[target_column]

In [22]:
# Training and testing data split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# check the shape of X_train and X_test
print(X_train.shape, X_test.shape)

(242, 13) (61, 13)


In [29]:
# Normalisation
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

In [31]:
# Calculate the input ranges for each feature
input_ranges = np.zeros((X.shape[1], 2))
for i, col in enumerate(X.columns):
    input_ranges[i, 0] = X[col].min()
    input_ranges[i, 1] = X[col].max()

In [30]:
import torch as th

class NAM(th.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1):
        super(NAM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.submodules = th.nn.ModuleList()
        print(input_dim)

        # Create the submodules for each input feature
        for i in range(input_dim):
            submodule = th.nn.Sequential()
            # Add layers to the submodule
            for l in range(num_layers):
                if l == 0:
                    submodule.add_module(f"linear_{l}", th.nn.Linear(1, hidden_dim))
                else:
                    submodule.add_module(f"linear_{l}", th.nn.Linear(hidden_dim, hidden_dim))
                submodule.add_module(f"ELU_{l}", th.nn.ELU())
                submodule.add_module(f"dropout_{l}", th.nn.Dropout(0.5))

            # Add the output layer
            submodule.add_module(f"linear_{num_layers}", th.nn.Linear(hidden_dim, output_dim))
            self.submodules.append(submodule)

    def forward(self, x):
        output = th.zeros(x.shape[0], self.output_dim)

        # Apply each submodule to its corresponding input feature and sum the results
        for i in range(self.input_dim):
            output += self.submodules[i](x[:, i].unsqueeze(1))

        # Apply sigmoid activation for binary classification
        return th.sigmoid(output)

    def init_weights(self, m):
        # Initialize the weights of linear layers
        if type(m) == th.nn.Linear:
            th.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def get_feature_maps(self, resolution=100):
        # Initialize the output
        output = th.zeros(self.input_dim, resolution)

        # For each input dimension, pass it through the corresponding submodule
        for i in range(self.input_dim):
            for j in range(resolution):
                # Add each input dimension output to the overall output
                output[i, j] = self.submodules[i](th.tensor([[j / resolution]]))

        # Return the overall output as a NumPy array
        return output.detach().numpy()

In [32]:
# Define the Neural Additive Model
# One input dimension for each feature
# One output dimension for binary classification, using sigmoid activation on the output layer

model = NAM(input_dim, hidden_dim=10, output_dim=1, num_layers=3)

# Set seeds for reproducibility
seed = 42
th.manual_seed(seed)
np.random.seed(seed)

# Initialize weights
model.apply(model.init_weights)
print(model)

13
NAM(
  (submodules): ModuleList(
    (0-12): 13 x Sequential(
      (linear_0): Linear(in_features=1, out_features=10, bias=True)
      (ELU_0): ELU(alpha=1.0)
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_1): Linear(in_features=10, out_features=10, bias=True)
      (ELU_1): ELU(alpha=1.0)
      (dropout_1): Dropout(p=0.5, inplace=False)
      (linear_2): Linear(in_features=10, out_features=10, bias=True)
      (ELU_2): ELU(alpha=1.0)
      (dropout_2): Dropout(p=0.5, inplace=False)
      (linear_3): Linear(in_features=10, out_features=1, bias=True)
    )
  )
)


In [34]:
# Define the optimizer
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Define the loss function (binary cross entropy for binary classification)
loss_fn = th.nn.BCELoss()

In [35]:
# Define the train function
def train(model, X, y, optimizer, loss_fn):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients
    X_tensor = th.tensor(X.to_numpy(), dtype=th.float32)  # Convert input data to tensor
    y_tensor = th.tensor(y.to_numpy(), dtype=th.float32).unsqueeze(1)  # Convert target data to tensor
    output = model(X_tensor)  # Forward pass to get the model output
    loss = loss_fn(output, y_tensor)  # Compute the loss using the loss function
    loss.backward()  # Compute the gradients through backpropagation
    optimizer.step()  # Update the model weights using the optimizer
    return loss.item()  # Return the loss value
# Define the evaluation function
def evaluate(model, X, y, loss_fn):
    model.eval()  # Set the model to evaluation mode
    X_tensor = th.tensor(X.to_numpy(), dtype=th.float32)  # Convert input data to tensor
    y_tensor = th.tensor(y.to_numpy(), dtype=th.float32).unsqueeze(1)  # Convert target data to tensor
    output = model(X_tensor)  # Forward pass to get the model output
    loss = loss_fn(output, y_tensor)  # Compute the loss using the loss function
    predictions = (output > 0.5).float()  # Threshold the output probabilities to binary predictions
    correct = (predictions == y_tensor).sum().item()  # Count the number of correct predictions
    accuracy = correct / y_tensor.shape[0]  # Compute the accuracy
    return loss.item(), accuracy  # Return the loss and accuracy

In [36]:
# Train the model
for epoch in range(1000):
    train_loss = train(model, X_train, y_train, optimizer, loss_fn)
    test_loss, test_accuracy = evaluate(model, X_test, y_test, loss_fn)
    print(f'Epoch {epoch + 1}: train loss = {train_loss:.4f}, test loss = {test_loss:.4f}, test accuracy = {test_accuracy:.4f}')

print(f'Final test accuracy: {test_accuracy:.4f}')

Epoch 1: train loss = 1.0273, test loss = 0.8726, test accuracy = 0.4590
Epoch 2: train loss = 1.0675, test loss = 0.8248, test accuracy = 0.4754
Epoch 3: train loss = 0.9585, test loss = 0.7865, test accuracy = 0.4754
Epoch 4: train loss = 0.9115, test loss = 0.7567, test accuracy = 0.5410
Epoch 5: train loss = 0.9313, test loss = 0.7349, test accuracy = 0.4590
Epoch 6: train loss = 0.9627, test loss = 0.7192, test accuracy = 0.5574
Epoch 7: train loss = 0.9740, test loss = 0.7085, test accuracy = 0.5574
Epoch 8: train loss = 0.8977, test loss = 0.7012, test accuracy = 0.5574
Epoch 9: train loss = 0.8935, test loss = 0.6959, test accuracy = 0.5246
Epoch 10: train loss = 1.0353, test loss = 0.6916, test accuracy = 0.5082
Epoch 11: train loss = 0.8622, test loss = 0.6873, test accuracy = 0.5082
Epoch 12: train loss = 0.9035, test loss = 0.6828, test accuracy = 0.5082
Epoch 13: train loss = 0.8589, test loss = 0.6776, test accuracy = 0.5246
Epoch 14: train loss = 0.9162, test loss = 0.67

Epoch 118: train loss = 0.5419, test loss = 0.4551, test accuracy = 0.7705
Epoch 119: train loss = 0.4849, test loss = 0.4544, test accuracy = 0.7705
Epoch 120: train loss = 0.4923, test loss = 0.4539, test accuracy = 0.7705
Epoch 121: train loss = 0.5855, test loss = 0.4534, test accuracy = 0.7705
Epoch 122: train loss = 0.5893, test loss = 0.4530, test accuracy = 0.7705
Epoch 123: train loss = 0.5574, test loss = 0.4526, test accuracy = 0.7705
Epoch 124: train loss = 0.5468, test loss = 0.4523, test accuracy = 0.7705
Epoch 125: train loss = 0.5279, test loss = 0.4520, test accuracy = 0.7705
Epoch 126: train loss = 0.6039, test loss = 0.4517, test accuracy = 0.7705
Epoch 127: train loss = 0.5610, test loss = 0.4514, test accuracy = 0.7705
Epoch 128: train loss = 0.5465, test loss = 0.4512, test accuracy = 0.7705
Epoch 129: train loss = 0.5907, test loss = 0.4510, test accuracy = 0.7705
Epoch 130: train loss = 0.5497, test loss = 0.4510, test accuracy = 0.7705
Epoch 131: train loss = 0

Epoch 232: train loss = 0.4933, test loss = 0.4248, test accuracy = 0.7705
Epoch 233: train loss = 0.5018, test loss = 0.4246, test accuracy = 0.7705
Epoch 234: train loss = 0.4598, test loss = 0.4243, test accuracy = 0.7705
Epoch 235: train loss = 0.4514, test loss = 0.4241, test accuracy = 0.7705
Epoch 236: train loss = 0.5368, test loss = 0.4239, test accuracy = 0.7705
Epoch 237: train loss = 0.4776, test loss = 0.4238, test accuracy = 0.7705
Epoch 238: train loss = 0.5165, test loss = 0.4237, test accuracy = 0.7705
Epoch 239: train loss = 0.5199, test loss = 0.4235, test accuracy = 0.7705
Epoch 240: train loss = 0.4230, test loss = 0.4235, test accuracy = 0.7705
Epoch 241: train loss = 0.4169, test loss = 0.4234, test accuracy = 0.7705
Epoch 242: train loss = 0.4365, test loss = 0.4232, test accuracy = 0.7705
Epoch 243: train loss = 0.4035, test loss = 0.4231, test accuracy = 0.7705
Epoch 244: train loss = 0.4808, test loss = 0.4229, test accuracy = 0.7705
Epoch 245: train loss = 0

Epoch 348: train loss = 0.4462, test loss = 0.4205, test accuracy = 0.7705
Epoch 349: train loss = 0.4396, test loss = 0.4200, test accuracy = 0.7705
Epoch 350: train loss = 0.4465, test loss = 0.4195, test accuracy = 0.7705
Epoch 351: train loss = 0.4522, test loss = 0.4192, test accuracy = 0.7705
Epoch 352: train loss = 0.4839, test loss = 0.4189, test accuracy = 0.7705
Epoch 353: train loss = 0.4425, test loss = 0.4186, test accuracy = 0.7705
Epoch 354: train loss = 0.4213, test loss = 0.4183, test accuracy = 0.7705
Epoch 355: train loss = 0.4519, test loss = 0.4179, test accuracy = 0.7705
Epoch 356: train loss = 0.4303, test loss = 0.4175, test accuracy = 0.7705
Epoch 357: train loss = 0.4596, test loss = 0.4171, test accuracy = 0.7705
Epoch 358: train loss = 0.4828, test loss = 0.4169, test accuracy = 0.7705
Epoch 359: train loss = 0.4055, test loss = 0.4166, test accuracy = 0.7705
Epoch 360: train loss = 0.5173, test loss = 0.4165, test accuracy = 0.7705
Epoch 361: train loss = 0

Epoch 460: train loss = 0.4170, test loss = 0.4050, test accuracy = 0.7705
Epoch 461: train loss = 0.4617, test loss = 0.4047, test accuracy = 0.7705
Epoch 462: train loss = 0.4293, test loss = 0.4046, test accuracy = 0.7705
Epoch 463: train loss = 0.4343, test loss = 0.4046, test accuracy = 0.7705
Epoch 464: train loss = 0.4106, test loss = 0.4047, test accuracy = 0.7705
Epoch 465: train loss = 0.3897, test loss = 0.4047, test accuracy = 0.7705
Epoch 466: train loss = 0.3950, test loss = 0.4048, test accuracy = 0.7705
Epoch 467: train loss = 0.4080, test loss = 0.4050, test accuracy = 0.7705
Epoch 468: train loss = 0.4522, test loss = 0.4053, test accuracy = 0.7705
Epoch 469: train loss = 0.4366, test loss = 0.4054, test accuracy = 0.7869
Epoch 470: train loss = 0.5015, test loss = 0.4054, test accuracy = 0.7869
Epoch 471: train loss = 0.4072, test loss = 0.4053, test accuracy = 0.7869
Epoch 472: train loss = 0.4616, test loss = 0.4048, test accuracy = 0.7869
Epoch 473: train loss = 0

Epoch 575: train loss = 0.4859, test loss = 0.4004, test accuracy = 0.7869
Epoch 576: train loss = 0.4763, test loss = 0.4005, test accuracy = 0.7869
Epoch 577: train loss = 0.4266, test loss = 0.4006, test accuracy = 0.7869
Epoch 578: train loss = 0.4527, test loss = 0.4005, test accuracy = 0.7869
Epoch 579: train loss = 0.4354, test loss = 0.4004, test accuracy = 0.7869
Epoch 580: train loss = 0.4111, test loss = 0.4004, test accuracy = 0.7869
Epoch 581: train loss = 0.4360, test loss = 0.4002, test accuracy = 0.7869
Epoch 582: train loss = 0.4408, test loss = 0.4000, test accuracy = 0.7869
Epoch 583: train loss = 0.4620, test loss = 0.3998, test accuracy = 0.7869
Epoch 584: train loss = 0.4238, test loss = 0.3998, test accuracy = 0.7869
Epoch 585: train loss = 0.4050, test loss = 0.4000, test accuracy = 0.7705
Epoch 586: train loss = 0.4284, test loss = 0.4002, test accuracy = 0.7705
Epoch 587: train loss = 0.4306, test loss = 0.4004, test accuracy = 0.7705
Epoch 588: train loss = 0

Epoch 692: train loss = 0.4078, test loss = 0.3852, test accuracy = 0.7869
Epoch 693: train loss = 0.3901, test loss = 0.3850, test accuracy = 0.7869
Epoch 694: train loss = 0.4037, test loss = 0.3847, test accuracy = 0.7869
Epoch 695: train loss = 0.3725, test loss = 0.3843, test accuracy = 0.7869
Epoch 696: train loss = 0.3912, test loss = 0.3839, test accuracy = 0.7869
Epoch 697: train loss = 0.4226, test loss = 0.3835, test accuracy = 0.7869
Epoch 698: train loss = 0.4386, test loss = 0.3833, test accuracy = 0.7869
Epoch 699: train loss = 0.4107, test loss = 0.3830, test accuracy = 0.7869
Epoch 700: train loss = 0.4312, test loss = 0.3829, test accuracy = 0.7869
Epoch 701: train loss = 0.4372, test loss = 0.3826, test accuracy = 0.7869
Epoch 702: train loss = 0.4325, test loss = 0.3824, test accuracy = 0.7869
Epoch 703: train loss = 0.4401, test loss = 0.3823, test accuracy = 0.7869
Epoch 704: train loss = 0.4111, test loss = 0.3823, test accuracy = 0.7869
Epoch 705: train loss = 0

Epoch 808: train loss = 0.4121, test loss = 0.3734, test accuracy = 0.8525
Epoch 809: train loss = 0.4170, test loss = 0.3736, test accuracy = 0.8525
Epoch 810: train loss = 0.3935, test loss = 0.3739, test accuracy = 0.8525
Epoch 811: train loss = 0.4179, test loss = 0.3740, test accuracy = 0.8361
Epoch 812: train loss = 0.3962, test loss = 0.3741, test accuracy = 0.8361
Epoch 813: train loss = 0.4383, test loss = 0.3742, test accuracy = 0.8361
Epoch 814: train loss = 0.4213, test loss = 0.3742, test accuracy = 0.8361
Epoch 815: train loss = 0.4397, test loss = 0.3741, test accuracy = 0.8361
Epoch 816: train loss = 0.3660, test loss = 0.3742, test accuracy = 0.8361
Epoch 817: train loss = 0.4027, test loss = 0.3744, test accuracy = 0.8361
Epoch 818: train loss = 0.4162, test loss = 0.3742, test accuracy = 0.8361
Epoch 819: train loss = 0.4256, test loss = 0.3740, test accuracy = 0.8361
Epoch 820: train loss = 0.4494, test loss = 0.3738, test accuracy = 0.8361
Epoch 821: train loss = 0

Epoch 923: train loss = 0.3567, test loss = 0.3690, test accuracy = 0.8525
Epoch 924: train loss = 0.3922, test loss = 0.3688, test accuracy = 0.8525
Epoch 925: train loss = 0.4042, test loss = 0.3688, test accuracy = 0.8525
Epoch 926: train loss = 0.4014, test loss = 0.3688, test accuracy = 0.8525
Epoch 927: train loss = 0.4136, test loss = 0.3690, test accuracy = 0.8525
Epoch 928: train loss = 0.4203, test loss = 0.3693, test accuracy = 0.8525
Epoch 929: train loss = 0.3737, test loss = 0.3694, test accuracy = 0.8525
Epoch 930: train loss = 0.3756, test loss = 0.3694, test accuracy = 0.8525
Epoch 931: train loss = 0.3896, test loss = 0.3697, test accuracy = 0.8525
Epoch 932: train loss = 0.4059, test loss = 0.3700, test accuracy = 0.8525
Epoch 933: train loss = 0.4090, test loss = 0.3703, test accuracy = 0.8525
Epoch 934: train loss = 0.4453, test loss = 0.3706, test accuracy = 0.8525
Epoch 935: train loss = 0.4370, test loss = 0.3707, test accuracy = 0.8525
Epoch 936: train loss = 0